## DERIVATIVE PRICING
MODULE 3 | LESSON 4


---



# **OBJECT-ORIENTED PROGRAMMING IN THE TRINOMIAL TREE**


|  |  |
|:---|:---|
|**Reading Time** |  90 minutes |
|**Prior Knowledge** | Binomial and Trinomial model pricing dynamics, Python basics|
|**Keywords** | Object-oriented programming  |


---<span style='color: transparent; font-size:1%'>All rights reserved WQU WorldQuant University QQQQ</span>

*By now, you know how to price options under a more complex setting such as the trinomial tree. Lesson 4, however, is not as focused on the conceptual part of derivative pricing. Instead, we take advantage of the medium-level complexity of the trinomial model in order to introduce the notion of object-oriented programming (OOP) in the context of derivative pricing. This is purely a programming skill but one you need to master in order to succeed in highly quantitative jobs.*

*Before going into more complex frameworks, let's take the opportunity now to revisit the previous pricing process under the trinomial model to build our own `TrinomialModel` class algorithm.* 

**Note:** In case you are not familiar with OOP (object-oriented programming), [you may want to check out this tutorial on YouTube.](https://www.youtube.com/watch?v=Ej_02ICOIgs)

The YouTube video tutorial is very long, as it thoroughly goes over the main features of OOP. In spite of its length, it can help to clarify any questions you may have about object-oriented programming.


## **1. Let's Start Coding!**

As always, start by importing the necessary libraries:

In [1]:
import numpy as np

In [2]:
class TrinomialModel(object):  # Here we start defining our 'class' --> Trinomial Model!
    # First, a method to initialize our `TrinomialModel` algorithm!
    def __init__(self, S0, r, sigma, mat):
        self.__s0 = S0
        self.__r = r
        self.__sigma = sigma
        self.__T = mat

    # Second, we build a method (function) to compute the risk-neutral probabilities!
    def __compute_probs(self):
        self.__pu = (
            (
                np.exp(self.__r * self.__h / 2)
                - np.exp(-self.__sigma * np.sqrt(self.__h / 2))
            )
            / (
                np.exp(self.__sigma * np.sqrt(self.__h / 2))
                - np.exp(-self.__sigma * np.sqrt(self.__h / 2))
            )
        ) ** 2
        self.__pd = (
            (
                -np.exp(self.__r * self.__h / 2)
                + np.exp(self.__sigma * np.sqrt(self.__h / 2))
            )
            / (
                np.exp(self.__sigma * np.sqrt(self.__h / 2))
                - np.exp(-self.__sigma * np.sqrt(self.__h / 2))
            )
        ) ** 2
        self.__pm = 1 - self.__pu - self.__pd

        assert 0 <= self.__pu <= 1.0, "p_u should lie in [0, 1] given %s" % self.__pu
        assert 0 <= self.__pd <= 1.0, "p_d should lie in [0, 1] given %s" % self.__pd
        assert 0 <= self.__pm <= 1.0, "p_m should lie in [0, 1] given %s" % self.__pm

    # Third, this method checks whether the given parameters are alright and that we have a 'recombining tree'!
    def __check_up_value(self, up):
        if up is None:
            up = np.exp(self.__sigma * np.sqrt(2 * self.__h))

        assert up > 0.0, "up should be non negative"

        down = 1 / up

        assert down < up, "up <= 1. / up = down"

        self.__up = up
        self.__down = down

    # Four, we use this method to compute underlying stock price path
    def __gen_stock_vec(self, nb):
        vec_u = self.__up * np.ones(nb)
        np.cumprod(vec_u, out=vec_u)

        vec_d = self.__down * np.ones(nb)
        np.cumprod(vec_d, out=vec_d)

        res = np.concatenate((vec_d[::-1], [1.0], vec_u))
        res *= self.__s0

        return res

    # Fifth, we declare a Payoff method to be completed afterwards depending on the instrument we are pricing!
    def payoff(self, stock_vec):
        raise NotImplementedError()

    # Sixth, compute current prices!
    def compute_current_price(self, crt_vec_stock, nxt_vec_prices):
        expectation = np.zeros(crt_vec_stock.size)
        for i in range(expectation.size):
            tmp = nxt_vec_prices[i] * self.__pd
            tmp += nxt_vec_prices[i + 1] * self.__pm
            tmp += nxt_vec_prices[i + 2] * self.__pu

            expectation[i] = tmp

        return self.__discount * expectation

    # Seventh, Option pricing!
    def price(self, nb_steps, up=None):
        assert nb_steps > 0, "nb_steps shoud be > 0"

        nb_steps = int(nb_steps)

        self.__h = self.__T / nb_steps
        self.__check_up_value(up)
        self.__compute_probs()

        self.__discount = np.exp(-self.__r * self.__h)

        final_vec_stock = self.__gen_stock_vec(nb_steps)
        final_payoff = self.payoff(final_vec_stock)
        nxt_vec_prices = final_payoff

        for i in range(1, nb_steps + 1):
            vec_stock = self.__gen_stock_vec(nb_steps - i)
            nxt_vec_prices = self.compute_current_price(vec_stock, nxt_vec_prices)

        return nxt_vec_prices[0]

Next, let's define the initialization and payoff methods for a new class (`TrinomialCall`) within the `TrinomialModel` we have just seen:

In [3]:
class TrinomialCall(TrinomialModel):
    def __init__(self, S0, r, sigma, mat, K):
        super(TrinomialCall, self).__init__(S0, r, sigma, mat)
        self.__K = K

    def payoff(self, s):
        return np.maximum(s - self.__K, 0.0)

We have defined two classes: `TrinomialModel` and `TrinomialCall`. These allow us to quickly initialize a trinomial model pricing algorithm for the payoff of a European call option. Please take your time in going over the previous chunk of code and make sure you understand it thoroughly.

Now, let's check how our algorithm performs on a simple example with:

- $S_0 = 100$
- $r=0\%$
- $σ = 30\%$
- $T=1$
- $K=90$

In [4]:
tree = TrinomialCall(100.0, 0.0, 0.3, 1.0, 90.0)

Here, the tree variable contains all the different info from the algorithm in `TrinomialCall` class.

How can we see what the price is for, let's say, $N=2$?

In [5]:
print(tree.price(2))

17.50162310051333


As you see, we got the same number for the price of the Call option as we got from our previous example.

Note that one great thing about object orientation is that now we can define main sub-classes within the `TrinomialModel` as we want, each with a different payoff (e.g., European put option, American, etc.)




## **2. Can You Do the Same for a New Class that is `TrinomialPut`?**

## **3. Conclusion**

You now know how to implement all the previous pricing techniques in a more formal way regarding programming. In the next module, we will introduce more formal mathematical notation in order to explore the inherent flaws of simple pricing models based on trees. This will be the first step toward building a more complex model that recognizes the real features of assets.

---
Copyright 2025 WorldQuant University. This
content is licensed solely for personal use. Redistribution or
publication of this material is strictly prohibited.
